In [4]:
import numpy as np
import pandas as pd

# --- 1. 数据加载与参数计算 ---

# 步骤 1.1: 定义5年历史年度回报率数据
# 数据来源：基于比亚迪(1211.HK)和中国移动(0941.HK)在2019-2024年末的H股复权收盘价计算得出。
# 周期为五个完整年度：2020, 2021, 2022, 2023, 2024。
returns_data = {
    '比亚迪': [
        4.503285,  # 2020
        0.267431,  # 2021
        -0.249058, # 2022
        -0.041144, # 2023
        0.125065   # 2024
    ],
    '中国移动': [
        -0.318919, # 2020
        0.054422,  # 2021
        0.108602,  # 2022
        0.258002,  # 2023
        0.117965   # 2024
    ]
}
years_of_return = ['2020', '2021', '2022', '2023', '2024']
df_returns = pd.DataFrame(returns_data, index=years_of_return)

# 步骤 1.2: 计算预期年化收益率向量 (r) 和协方差矩阵 (Σ)
# 预期收益率 (r) 是历史年回报率的算术平均值。
r = df_returns.mean()

# 协方差矩阵 (Σ) 衡量了资产回报率自身的波动性（方差）以及资产之间的联动性（协方差）。
Sigma = df_returns.cov()


# --- 2. 投资组合构建与核心量化分析 ---

# 步骤 2.1: 创建一个代表等权重（50%, 50%）配置的权重向量 (w)。
w = np.array([0.5, 0.5])

# 步骤 2.2: 计算投资组合的预期总回报。
# 金融含义：通过向量点积 (w^T * r) 计算组合的加权平均回报。
# 这是组合内各资产预期回报按其投资权重的加权总和。
portfolio_return = np.dot(w.T, r)

# 步骤 2.3: 计算整个投资组合的整体风险（年化波动率）。
# 金融含义：通过二次型运算 (w^T * Σ * w) 计算组合方差。
# 这个运算不仅考虑了各资产自身的风险（方差），还通过协方差项纳入了资产间的相互影响。
# 负协方差会降低整体风险，起到分散化的作用。
portfolio_variance = np.dot(w.T, np.dot(Sigma, w))
portfolio_risk = np.sqrt(portfolio_variance)


# --- 3. 结果输出 ---

print("=" * 50)
print("             投资组合量化分析报告")
print("=" * 50)

print("\n【输入数据】")
print("\n1. 5年历史年度回报率 (Historical Annual Returns):")
print(df_returns.to_string(float_format="%.4f"))

print("\n\n【核心参数计算结果】")
print("\n2. 预期年化收益率向量 (Expected Annualized Return Vector, r):")
print(r.to_string(float_format="%.4f"))

print("\n3. 协方差矩阵 (Covariance Matrix, Σ):")
print(Sigma.to_string(float_format="%.4f"))

print("\n\n【等权重投资组合 (50/50) 分析结果】")
print("\n4. 权重向量 (Weight Vector, w):")
print(w)

print(f"\n5. 投资组合预期年回报率 (Portfolio Expected Annual Return): {portfolio_return:.2%}")
print(f"6. 投资组合年化风险/波动率 (Portfolio Annualized Risk/Volatility): {portfolio_risk:.2%}")

print("\n" + "=" * 50)

             投资组合量化分析报告

【输入数据】

1. 5年历史年度回报率 (Historical Annual Returns):
         比亚迪    中国移动
2020  4.5033 -0.3189
2021  0.2674  0.0544
2022 -0.2491  0.1086
2023 -0.0411  0.2580
2024  0.1251  0.1180


【核心参数计算结果】

2. 预期年化收益率向量 (Expected Annualized Return Vector, r):
比亚迪    0.9211
中国移动   0.0440

3. 协方差矩阵 (Covariance Matrix, Σ):
         比亚迪    中国移动
比亚迪   4.0470 -0.4118
中国移动 -0.4118  0.0468


【等权重投资组合 (50/50) 分析结果】

4. 权重向量 (Weight Vector, w):
[0.5 0.5]

5. 投资组合预期年回报率 (Portfolio Expected Annual Return): 48.26%
6. 投资组合年化风险/波动率 (Portfolio Annualized Risk/Volatility): 90.42%



1.我们今天使用了固定的“等权重”配置。在真实的基金管理中，这是一个好的策略吗？如果不是，您会如何设计一个流程，去寻找那个“最优”的权重组合？

2.我们的所有计算都基于“历史数据”。这在金融市场中存在一个巨大的风险，它叫什么？（提示：与“黑天鹅”有关）。作为决策者，您会如何应对这种风险？

.这个“回报-风险”分析框架，还能应用在哪些您MBA课程中涉及的非金融领域？（例如，一家公司在考虑进入三个新市场，每个市场有不同的预期回报和风险，且市场之间存在一定的相关性，您会如何为CEO提供决策建议？）

当然。这三个问题触及了从量化分析到战略决策的核心，是MBA课程中非常关键的思维跃迁。我会逐一为您解答。

---

### 1. 关于投资组合权重：超越“等权重”的优化策略

在真实的基金管理中，**“等权重”通常被视为一种“朴素”或“无信息”的策略**。它简单、易于执行，并且能避免将所有赌注压在单一资产上。

**但它远非最优。** 原因是它完全忽略了我们辛苦计算出的核心参数：
* **它无视预期回报的差异**：给予高预期回报的资产（比亚迪）和低预期回报的资产（中国移动）相同的权重。
* **它无视风险水平的差异**：给予高风险资产（比亚迪）和低风险资产（中国移动）相同的权重。
* **它无视资产间的相关性**：虽然被动地从负相关性中获益，但并未主动利用这一特性去构建风险调整后收益最佳的组合。

#### **寻找“最优”权重组合的流程（基于现代投资组合理论MPT）**

我会设计一个包含四个步骤的系统化流程：

**第一步：生成“可行集”与“有效前沿” (Efficient Frontier)**
* **流程**：通过代码模拟成千上万种不同的权重组合（例如，比亚迪0%/移动100%，比亚迪1%/移动99%...以此类推），并为每一种组合计算其对应的预期回报率和风险（波动率）。
* **产出**：将所有这些组合的回报-风险坐标点绘制成图，形成一个“可行集”。其中，在每个特定风险水平上能够提供最高回报的点的连线，就是**“有效前沿”**。
* **金融含义**：任何位于“有效前沿”下方的投资组合都是“次优”的，因为在相同风险下，总能找到一个回报更高的组合。理性投资者只应在“有效前沿”上做选择。

**第二步：引入“夏普比率”，锁定“最优风险组合” (Optimal Risky Portfolio)**
* **流程**：计算“有效前沿”上每一个组合的**夏普比率（Sharpe Ratio）**。
    * 夏普比率 = (投资组合预期回报率 - 无风险利率) / 投资组合风险
* **产出**：找到那个使夏普比率最大化的投资组合。在图形上，它表现为从无风险利率点出发与“有效前沿”相切的那个切点。
* **金融含义**：夏普比率衡量的是**“每承担一单位风险，能获得多少超越无风险利率的超额回报”**。它是衡量投资性价比的黄金标准。夏普比率最高的组合，是纯粹由风险资产构成的“数学最优解”，因为它提供了最佳的风险调整后收益。

**第三步：结合投资者个人风险偏好，构建最终组合 (Capital Allocation Line)**
* **流程**：将上一步找到的“最优风险组合”与**无风险资产**（如国债）进行结合。
* **产出**：根据投资者的风险偏好（保守、中性、激进）来决定两者的配置比例。
    * **保守型投资者**：可能配置30%的“最优风险组合”和70%的无风险资产。
    * **激进型投资者**：可能配置100%甚至通过杠杆配置120%的“最优风险组合”。
* **金融含义**：这个流程将数学上的“最优解”与投资者的“个性化需求”完美结合，最终形成一个既科学又符合个人情况的投资决策。

---

### 2. 关于历史数据的风险：“黑天鹅”与“尾部风险”

是的，所有基于历史数据的量化模型都存在一个巨大的、甚至可能是致命的风险。这个风险被称为 **“尾部风险” (Tail Risk)**。

“**黑天鹅**”指的是超出正常预期范围、极其罕见、一旦发生会带来颠覆性影响的事件。而“**尾部风险**”就是指这类“黑天鹅”事件发生的风险。

**为什么依赖历史数据是危险的？**
因为历史数据，尤其是较短时期的数据，往往会呈现一种“伪常态”。模型基于这些数据，会认为未来的波动范围也大致如此。但“黑天鹅”事件的特点恰恰是**“史无前例”**。例如，没有任何历史数据可以预测到2008年的金融海啸或2020年的全球疫情的规模和影响。过度依赖历史数据的模型，在“黑天鹅”面前会彻底失效。

#### **作为决策者，我会如何应对这种风险？**

应对“尾部风险”不能依靠单一工具，而是一个立体的风险管理体系：

1.  **进行压力测试与情景分析**：
    * 不满足于历史数据计算出的那个协方差矩阵。我会主动设定极端情景，例如：“假设中美贸易摩擦升级，导致两家公司的相关性从负值瞬间变为强正值，我们的组合会亏损多少？”“假设比亚迪供应链出现重大中断，其股价下跌80%，对组合的影响如何？” 这就是从“预测未来”转向“为未来做准备”。

2.  **定性分析与定量模型相结合**：
    * 绝不盲信模型输出。我会将量化结果作为“基础参考”，但最终决策必须融入对**宏观环境、产业政策、公司治理、技术壁垒、管理层能力**等基本面因素的深度定性分析。定量模型回答“过去是什么样”，而定性分析回答“未来可能会发生什么结构性变化”。

3.  **在投资组合中构建“反脆弱性”**：
    * 除了传统的股债配置，我会加入少量能够从混乱和波动中受益的资产。例如，购买**看跌期权**作为保险，或者配置少量**黄金、比特币**等与主流金融体系关联度低的资产。持有充足的**现金**本身也是一种反脆弱策略，因为它能在市场崩溃时提供最佳的购买力。

---

### 3. “回报-风险”框架的跨领域应用：新市场进入决策

您提出的这个例子，完美地展示了该框架的普适性。我会这样为CEO提供决策建议：

**第一步：将金融概念转化为商业语言**

* **预期回报** -> **预估的战略价值/投资回报率(ROI)**：可以是新市场的预期利润、市场份额、或对公司品牌、技术的战略提升。
* **风险 (波动率)** -> **商业不确定性**：可以是实现预期ROI的不确定性。例如，市场A的预期ROI是15%，但实现概率高，波动范围可能只有±3%；市场B预期ROI高达30%，但政局不稳、竞争激烈，波动范围可能是±20%。
* **相关性/协方差** -> **市场间的联动性**：
    * **正相关**：例如，同时进入美国和欧洲的高端消费品市场。这两个市场都受全球奢侈品消费周期的共同影响。一荣俱荣，一损俱损。
    * **负相关**：例如，一个市场是高端旅游（经济好时增长），另一个是廉价快消品（经济差时需求反而上升）。这两个市场的业务表现可以相互对冲。

**第二步：构建“企业资源配置的有效前沿”**

1.  **量化选项**：为进入三个新市场（A, B, C）的每一个选项，都量化出其“预期战略回报” (r_A, r_B, r_C) 和“商业不确定性” (σ_A, σ_B, σ_C)。
2.  **分析联动性**：评估市场间的“协方差矩阵”。进入A市场是否会促进B市场的成功（负协方差）？A和B是否共享同一个脆弱的供应链（正协方差）？
3.  **绘制决策地图**：像构建投资组合一样，模拟不同的**资源分配权重**（例如，方案一：50%资源给A，30%给B，20%给C；方案二：80%资源给A，20%给C...），计算每种方案的**“总体战略回报”**和**“总体商业风险”**。
4.  **提供“有效前沿”**：向CEO展示一张决策图。图的横轴是总体风险，纵轴是总体回报。“有效前沿”上的每一个点，都代表一种资源分配方案，它们是在特定风险水平下，公司能得到的最佳回报。

**最终建议：**
我的建议不会是“我们应该进入A市场”，而是：
“CEO，根据我们的分析，公司面临一个**战略选择的‘有效前沿’**。方案甲（重仓A）最稳健，预期回报12%，总体风险低。方案乙（重仓B）最激进，预期回报25%，但风险极高。而**方案丙（A市场占60%，C市场占30%，B市场占10%）位于‘有效前沿’的切点，提供了最佳的风险回报性价比。** 这套方案兼顾了增长与稳定，并且利用了C市场对A、B市场的对冲效应。这与我们公司当前‘追求稳健增长’的战略目标最为契合。”

通过这个框架，我们将一个复杂的、看似凭感觉的战略问题，转化成了一个**有数据支撑、有多种备选、且与公司风险偏好相匹配的科学决策过程**。